In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
# make graph function
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

### Question 1: Use yfinance to Extract (TESLA) Stock Data

In [7]:
tesla_data = yf.Ticker('TSLA')    # extraction of data

tesla_data = tesla_data.history(period='max')   # period parameter to max

tesla_data.reset_index(inplace=True)    # index reset

tesla_data.head()   # The first five rows

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


### Question 2: Use Webscraping to Extract Tesla Revenue Data

In [71]:
tesla_url="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"

html_data = requests.get(tesla_url)

soup = BeautifulSoup(html_data.text, 'html')

In [72]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue']) # Data frame

table_row = soup.find_all('tbody')[1].find_all('tr')

for row in table_row:
  table_data = row.find_all('td')
  record = [data.text for data in table_data]
  
  #Add to DataFrame
  tesla_revenue.loc[len(tesla_revenue)] = record

In [73]:
#Remove comma and dollar sign
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace('$', "")
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace(',', "")

# remove null or empty string
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


### Question 3: Use yfinance to Extract (GameStop) Stock Data

In [52]:
gme_data = yf.Ticker('GME') # extracting data

gme_data = gme_data.history(period='max')   # period parameter to max

gme_data.reset_index(inplace=True)  # Reset index

gme_data.head()   # Display first five rows

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4: Use Webscraping to Extract GME Revenue Data

In [75]:
gme_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'

gme_html_data = requests.get(gme_url).text  #Extracting data

gme_soup = BeautifulSoup(gme_html_data, 'html') #parsing data to beautiful soup


In [76]:
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue']) # Data frame

gme_table_rows = gme_soup.find_all('tbody')[1].find_all('tr') #

for row in gme_table_rows:
  gme_table_data = row.find_all('td') # all table data'td'
  result = [data.text for data in gme_table_data]
  
  # Add to Df
  gme_revenue.loc[len(gme_revenue)] = result

# remove comma and dollar symbol
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace('$', "") 
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace(',', "")
  
# remove null or empty string
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

gme_revenue.tail()  # the last five rows

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


### Question 5: Plot Tesla Stock Graph

In [ ]:
make_graph(tesla_data, tesla_revenue, "Tesla")

0e345936d8d4c1ed890eedd7d6d4bc01

https://eu-de.dataplatform.cloud.ibm.com/analytics/notebooks/v2/3bdb1f4d-d440-41ce-b0de-b19eead1c44d/view?access_token=4a2aa491ec9e55b845ab819f6b93b5387f933640848eee66aea67452cc83dff5&context=cpdaas